# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 95.8%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.98,0.92,0.95,0.98,0.98,0.95,0.96,0.98,0.98,0.95,0.98,0.96,0.98,0.97,0.95,0.88,0.97,0.98,0.88,0.98,0.98,0.95,0.96,0.96,0.97,0.88,0.98,0.89,0.97,0.93,0.97,0.98,0.99,0.92,0.98,0.90,0.98,0.99,0.89,0.96,0.98,0.98,0.98,0.98,0.98,0.98,0.97,0.91,0.98,0.98,0.98,0.93,0.98,0.98,0.91,0.91,0.84,0.93,0.98,0.98,0.98,0.98,0.94,0.98,0.98,0.93,0.97,0.98,0.98,0.98,0.95,0.97,0.96,0.98,0.94,0.98,0.94,0.98,0.98,0.95


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.96
2005,0.93
2006,0.94
2007,0.96
2008,0.97
2009,0.97
2010,0.97
2011,0.96


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.95
3,0.95
4,0.96
5,0.96
6,0.96
7,0.96
8,0.96
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.98
     2          0.98
     3          0.98
     4          0.98
     5          0.97
     6          0.95
     7          0.94
     8          0.96
     9          0.99
     10         0.99
     11         0.99
     12         0.97
2004 1          0.94
     2          0.93
     3          0.93
     4          0.96
     5          0.97
     6          0.97
     7          0.99
     8          0.98
     9          0.97
     10         0.95
     11         0.94
     12         0.93

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.47,0.43,0.46,0.47,0.47,0.45,0.46,0.47,0.47,0.46,0.47,0.46,0.47,0.46,0.46,0.42,0.47,0.47,0.42,0.46,0.47,0.45,0.46,0.46,0.46,0.42,0.47,0.42,0.46,0.45,0.46,0.47,0.47,0.43,0.47,0.43,0.47,0.47,0.43,0.46,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.44,0.46,0.47,0.47,0.44,0.47,0.47,0.44,0.44,0.39,0.44,0.47,0.47,0.47,0.47,0.45,0.46,0.47,0.44,0.46,0.47,0.47,0.47,0.45,0.46,0.45,0.47,0.45,0.46,0.45,0.47,0.47,0.45


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.46
2006,0.44
2007,0.49
2008,0.50
2009,0.46
2010,0.41
2011,0.48


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.50
4,0.41
5,0.39
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.36
     3          0.37
     4          0.53
     5          0.34
     6          0.30
     7          0.29
     8          0.29
     9          0.28
     10         0.43
     11         0.55
     12         0.54
2004 1          0.53
     2          0.50
     3          0.49
     4          0.39
     5          0.29
     6          0.38
     7          0.27
     8          0.41
     9          0.58
     10         0.55
     11         0.47
     12         0.47

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.98
2005,1.02
2006,1.00
2007,1.00
2008,1.00
2009,1.00
2010,1.00
2011,0.99


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.92
2,0.98
3,1.05
4,1.03
5,1.01
6,1.02
7,0.97
8,1.02
9,1.01


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.81
     2                 1.22
     3                 1.00
     4                 1.00
     5                 0.98
     6                 0.99
     7                 0.96
     8                 1.11
     9                 0.97
     10                1.00
     11                1.00
     12                0.97
2004 1                 0.63
     2                 0.84
     3                 1.15
     4                 1.35
     5                 1.12
     6                 1.19
     7                 1.03
     8                 1.01
     9                 0.92
     10                0.99
     11                0.79
     12                1.05

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $486,108.74/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,998,274.42","95,488,041.67"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"7,024,414.93"
2004,"13,549,503.37"
2005,"20,087,765.63"
2006,"12,903,117.04"
2007,"14,840,803.14"
2008,"12,274,305.72"
2009,"7,879,485.82"
2010,"7,488,986.42"
2011,"8,560,000.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,010.42","543,010.42","543,010.42","116,778.53","3,981,430.56"
2,"496,125.00","496,125.00","496,125.00","339,283.15","5,610,430.56"
3,"541,625.00","541,625.00","541,625.00","320,355.61","8,260,565.97"
4,"525,000.00","525,000.00","525,000.00","38,000.00","15,447,711.81"
5,"542,500.00","542,500.00","542,500.00",0.00,"9,516,300.35"
6,"525,000.00","525,000.00","525,000.00","27,290.18","6,164,062.50"
7,"542,500.00","542,500.00","542,500.00","247,000.00","5,136,611.11"
8,"542,500.00","542,500.00","542,500.00",0.00,"12,054,390.62"
9,"525,000.00","525,000.00","525,000.00","209,910.75","12,358,928.82"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00         608,196.18  
     8                   54,250.00                  0.00       4,498,437.50  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00  
2004 1                   54,250.00                  0.00               0.00  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00       4,029,907.99  
     4                   52,500.00                  0.00       1,565,625.00  
     5                   54,250.00                  0.00       1,276,239.58  
     6                   52,500.00             27,290.18       4,214,062.50  
     7                   54,250.00            247,000.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00             20,878.12               0.00  
     12                  54,250.00            247,000.00               0.00

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.91,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.00,0.94
2004,1.00,1.00,1.00,0.93,0.97
2005,1.00,1.00,1.00,0.00,0.97
2006,1.00,1.00,1.00,0.80,0.96
2007,1.00,1.00,1.00,0.95,0.97
2008,1.00,1.00,1.00,0.94,0.93
2009,1.00,1.00,1.00,0.94,0.97
2010,1.00,1.00,1.00,0.93,0.97
2011,1.00,1.00,1.00,0.00,0.97


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,117,700.52"
Repair,"23,098,221.35"
Crew Transfer,"1,215,140.63"
Site Travel,0.00
Mobilization,"8,500,000.00"
Weather Delay,"37,904,696.18"
No Requests,"21,796,133.98"
Not in Shift,"22,034,204.68"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,117,700.52","2,117,700.52"
Repair,0.00,0.00,0.00,"23,098,221.35","23,098,221.35"
Crew Transfer,0.00,0.00,0.00,"1,215,140.63","1,215,140.63"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"8,500,000.00","8,500,000.00"
Weather Delay,0.00,0.00,0.00,"37,904,696.18","37,904,696.18"
No Requests,0.00,0.00,0.00,"21,796,133.98","21,796,133.98"
Not in Shift,0.00,0.00,0.00,"22,034,204.68","22,034,204.68"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      62,416.67
     Repair        2,214,049.48
     Crew Transfer    97,463.54
     Site Travel           0.00
     Mobilization    500,000.00
     Weather Delay   872,013.02
     No Requests   1,449,847.22
     Not in Shift  1,828,625.00
2004 Maintenance     421,385.42
     Repair        2,243,566.84

In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                    
1     Maintenance     123,028.65
      Repair          679,583.33
      Crew Transfer    54,661.46
      Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 2,951,872.40
      No Requests     730,143.23
      Not in Shift    687,951.27
2     Maintenance     172,630.21
      Repair        1,502,407.12

In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          25,393.23
           Crew Transfer    2,934.90
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   23,315.10
           No Requests    100,460.94
           Not in Shift    12,177.08
     2     Maintenance          0.00
           Repair          20,544.27
           Crew Transfer    2,679.69
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   13,781.25
           No Requests    101,080.73
           Not in Shift     8,914.06
     3     Maintenance          0.00
           Repair          22,567.71
           Crew Transfer    2,697.92
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"124,682,578.98"


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   59,838,516.47   
          maintenance            0.00              0.00    2,117,700.52   
          repair                 0.00              0.00   23,098,221.35   

                         total_cost  
component action                     
turbine   delay       59,838,516.47  
          maintenance  2,117,700.52  
          repair      23,098,221.35

In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"2,844,000.00",0.00,"5,066,911.46","7,910,911.46"
2004,turbine,"4,836,500.00",0.00,"8,640,179.94","13,476,679.94"
2005,turbine,"5,234,000.00",0.00,"17,677,705.73","22,911,705.73"
2006,turbine,"5,030,000.00",0.00,"10,986,669.12","16,016,669.12"
2007,turbine,"4,059,000.00",0.00,"11,702,071.18","15,761,071.18"
2008,turbine,"4,085,500.00",0.00,"7,765,253.64","11,850,753.64"
2009,turbine,"2,797,500.00",0.00,"4,685,767.07","7,483,267.07"
2010,turbine,"2,959,000.00",0.00,"4,091,893.23","7,050,893.23"
2011,turbine,"2,644,000.00",0.00,"7,099,778.65","9,743,778.65"


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"1,572,500.00",0.00,"4,488,412.33","6,060,912.33"
2,turbine,"2,810,000.00",0.00,"6,203,882.98","9,013,882.98"
3,turbine,"2,999,500.00",0.00,"7,283,363.43","10,282,863.43"
4,turbine,"5,701,000.00",0.00,"14,667,045.14","20,368,045.14"
5,turbine,"3,713,000.00",0.00,"5,697,079.86","9,410,079.86"
6,turbine,"2,312,000.00",0.00,"3,669,907.37","5,981,907.37"
7,turbine,"2,519,500.00",0.00,"3,309,490.45","5,828,990.45"
8,turbine,"5,223,500.00",0.00,"11,571,514.76","16,795,014.76"
9,turbine,"4,978,500.00",0.00,"11,382,028.50","16,360,528.50"


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         43,500.00              0.00       62,289.06   
     2     turbine         17,000.00              0.00       43,294.27   
     3     turbine         53,000.00              0.00       36,130.21   
     4     turbine         19,000.00              0.00       35,875.00   
     5     turbine         66,500.00              0.00       28,729.17   
     6     turbine         80,500.00              0.00       40,687.50   
     7     turbine         60,000.00              0.00      147,024.31   
     8     turbine      2,042,500.00              0.00    4,372,902.78   
     9     turbine         69,500.00              0.00       45,463.54   
     10    turbine         88,000.00              0.00       68,888.02   
     11    turbine        108,500.00              0.00       55,908.85   
     12    turbine        196,000.00              0.00      129,718.75   
2004 1     turbine        221,500.00              0.00      161,000.00   
     2     turbine        124,000.00              0.00      149,625.00   
     3     turbine        956,000.00              0.00    3,692,657.99   
     4     turbine        886,500.00              0.00    1,721,375.00   
     5     turbine        602,000.00              0.00      788,250.00   
     6     turbine        659,500.00              0.00    1,235,363.10   
     7     turbine        294,500.00              0.00      125,093.75   
     8     turbine        172,000.00              0.00      123,302.08   
     9     turbine        124,000.00              0.00       99,203.13   
     10    turbine        221,500.00              0.00      149,807.29   
     11    turbine        147,500.00              0.00      167,367.19   
     12    turbine        427,500.00              0.00      227,135.42   
2005 1     turbine        499,500.00              0.00    3,644,180.56   
     2     turbine        838,000.00              0.00    2,742,312.50   
     3     turbine        213,500.00              0.00      151,502.60   
     4     turbine        129,000.00              0.00       84,473.96   

                       total_cost  
year month component               
2003 1     turbine     105,789.06  
     2     turbine      60,294.27  
     3     turbine      89,130.21  
     4     turbine      54,875.00  
     5     turbine      95,229.17  
     6     turbine     121,187.50  
     7     turbine     207,024.31  
     8     turbine   6,415,402.78  
     9     turbine     114,963.54  
     10    turbine     156,888.02  
     11    turbine     164,408.85  
     12    turbine     325,718.75  
2004 1     turbine     382,500.00  
     2     turbine     273,625.00  
     3     turbine   4,648,657.99  
     4     turbine   2,607,875.00  
     5     turbine   1,390,250.00  
     6     turbine   1,894,863.10  
     7     turbine     419,593.75  
     8     turbine     295,302.08  
     9     turbine     223,203.13  
     10    turbine     371,307.29  
     11    turbine     314,867.19  
     12    turbine     654,635.42  
2005 1     turbine   4,143,680.56  
     2     turbine   3,580,312.50  
     3     turbine     365,002.60  
     4     turbine     213,473.96

## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"199,749.22","68,815.75","62,454.50",485.00
major repair,"83,133.71","1,979.20","1,454.85",28.00
major replacement,"188,674.46","11,962.10","188,674.46",67.00
manual reset,"191,046.77","36,375.50","13,886.50","5,703.00"
medium repair,"13,212.15","10,707.75","8,583.75",214.00
minor repair,"65,893.61","31,159.50","15,561.50","2,236.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,613,900,001.00"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,613,900,001.00","122,699,492.00","114,177,882.50","120,589,493.50","122,868,866.00","123,608,760.00","117,117,852.50","120,954,834.00","123,054,093.00","123,207,752.50","119,706,382.50","123,192,978.50","120,599,006.00","123,214,736.00","122,220,699.50","119,851,003.00","110,296,620.00","122,540,066.00","122,509,605.00","109,955,693.00","122,232,828.00","122,868,219.00","118,106,319.50","120,391,110.50","119,741,724.50","121,000,561.50","111,538,380.00","123,405,535.50","109,459,251.00","121,798,692.00","117,839,094.50","122,273,391.00","122,984,384.00","124,135,738.50","113,692,548.50","122,862,923.00","113,873,926.50","122,826,199.00","124,584,746.50","112,130,106.00","120,153,767.50","123,316,210.00","123,203,184.00","123,137,880.50","123,451,840.00","123,583,652.00","122,649,075.00","122,476,853.50","114,477,291.50","122,253,570.00","122,828,838.00","123,102,097.00","116,293,741.00","123,311,130.50","122,621,567.00","115,524,542.50","115,203,067.50","103,560,206.50","116,091,064.50","123,586,630.00","122,871,849.50","123,573,608.50","123,135,753.00","117,414,391.50","121,762,985.00","122,926,591.50","116,077,209.50","120,438,175.00","122,650,071.50","122,490,470.00","123,156,112.00","118,488,364.00","120,939,134.50","119,668,502.50","123,594,042.50","118,397,000.00","121,948,701.00","118,784,001.00","123,729,292.00","123,025,084.00","117,890,959.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"861,243,958.00"
2004,"933,584,055.50"
2005,"957,000,053.00"
2006,"925,940,758.50"
2007,"1,023,918,305.50"
2008,"1,060,742,289.50"
2009,"967,397,252.50"
2010,"870,840,346.50"
2011,"1,017,665,442.50"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,069,257,066.00"
2,"810,649,279.00"
3,"891,844,889.00"
4,"705,709,397.50"
5,"691,173,641.00"
6,"557,848,516.00"
7,"584,028,442.00"
8,"640,075,456.50"
9,"793,890,237.50"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     110,680,024.50
     2      58,608,201.50
     3      66,867,495.50
     4      91,588,583.50
     5      60,973,711.50
     6      51,492,096.00
     7      50,999,367.50
     8      52,279,342.00
     9      47,813,381.50
     10     77,521,597.00
     11     95,604,975.50
     12     96,815,182.00
2004 1      94,552,237.50
     2      82,780,185.00
     3      87,581,044.00
     4      66,766,023.00
     5      51,982,625.00
     6      66,004,814.00
     7      49,054,003.50
     8      73,260,971.50
     9      99,541,955.00
     10     97,609,603.00
     11     80,465,511.50
     12     83,985,082.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [55]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

Exception: Runtime error: get_cf_project_return_aftertax_npv called for SAM_Singleowner but "cf_project_return_aftertax_npv" not assigned
